# Deep Learning from Pre-Trained Models with Keras

## Introduction

ImageNet, an image recognition benchmark dataset*, helped trigger the modern AI explosion.  In 2012, the AlexNet architecture (a deep convolutional-neural-network) rocked the ImageNet benchmark competition, handily beating the next best entrant.  By 2014, all the leading competitors were deep learning based.  Since then, accuracy scores continued to improve, eventually surpassing human performance.

In this hands-on tutorial we will build on this pioneering work to create our own neural-network architecture for image recognition.  Participants will use the elegant Keras deep learning programming interface to build and train TensorFlow models for image classification tasks on the CIFAR-10 / MNIST datasets*.  We will demonstrate the use of transfer learning* (to give our networks a head-start by building on top of existing, ImageNet pre-trained, network layers*), and explore how to improve model performance for standard deep learning pipelines.  We will use cloud-based interactive Jupyter notebooks to work through our explorations step-by-step.  Once participants have successfully trained their custom model we will show them how to submit their model's predictions to Kaggle for scoring*.

This tutorial aims to prepare participants for the HPC Saudi 2020 Student AI Competition.

Participants are expected to bring their own laptops and sign-up for free online cloud services (e.g., Google Colab, Kaggle).  They may also need to download free, open-source software prior to arriving for the workshop.

This tutorial assumes some basic knowledge of neural networks. If you’re not already familiar with neural networks, then you can learn the basics concepts behind neural networks at [course.fast.ai](https://course.fast.ai/).

* Tutorial materials are derived from:
  * [PyTorch Tutorials](https://github.com/kaust-vislab/pytorch-tutorials) by David Pugh.
  * [What is torch.nn really?](https://pytorch.org/tutorials/beginner/nn_tutorial.html) by Jeremy Howard, Rachel Thomas, Francisco Ingham.
  * [Machine Learning Notebooks](https://github.com/ageron/handson-ml2) (2nd Ed.) by Aurélien Géron.
  * *Deep Learning with Python* by François Chollet.

### Jupyter Notebooks

This is a Jupyter Notebook.  It provides a simple, cell-based, IDE for developing and exploring complex ideas via code, visualizations, and documentation.

A notebook has two primary types of cells: i) `markdown` cells for textual notes and documentation, such as the one you are reading now, and ii) `code` cells, which contain snippets of code (typically *Python*, but also *bash* scripts) that can be executed.  

The currently selected cell appears within a box. A green box indicates that the cell is editable.  Clicking inside a *code* cell makes it selected and editable.  Double-click inside *markdown* cells to edit.

Use `Tab` for context-sensitive code-completion assistance when editing Python code in *code* cells.  For example, use code assistance after a `.` seperator to find available object members.  For help documentation, create a new *code* cell, and use commands like `dir(`*module*`)`, `help(`*topic*`)`, `?`*name*, or `??`*function* for user provided *module*, *topic*, variable *name*, or *function* name.  The magic `?` and `??` commands show documentation / source code in a separate pane.

Clicking on `[Run]` or pressing `Ctrl-Enter` will execute the contents of a cell.  A *markdown* cell converts to its display version, and a *code* cell runs the code inside.  To the left of a *code* cell is a small text bracket `In [ ]:`.  If the bracket contains an asterix, e.g., `In [*]:`, that cell is currently executing.  Only one cell executes at a time (if multiple cells are *Run*, they are queued up to execute in the order they were run).  When a *code* cell finishes executing, the bracket shows an execution count in the bracket – each *code* cell execution increments the counter and provides a way to determine the order in which codes were executed – e.g., `In [7]` for the seventh cell to complete.  

The output produced by a *code* cell appears at the bottom of that cell after it executes.  The output generated by a code cell includes anything printed to the output during execution (e.g., print statements, or thrown errors) and the final value generated by the cell (i.e., not the intermediate values).  The final value is 'pretty printed' by Jupyter.

Typically, notebooks are written to be executed in order, from top to bottom.  Behind the scenes, however, each Notebook has a single Python state (the `kernel`), and each *code* cell that executes, modifies that state.  It is possible to modify and re-run earlier cells; however, care must be taken to also re-run any other cells that depend upon the modified one.  List the Python state global variables with the magic command `%wgets`.  The *kernel* can be restarted to a known state, and cell output cleared, if the Python state becomes too confusing to fix manually (choose `Restart & Clear Output` from the Jupyter `Kernel` menu) – this requires running each *code* cell again.

Complete user documentation is available at [jupyter-notebook.readthedocs.io](https://jupyter-notebook.readthedocs.io/en/stable/notebook.html#notebook-user-interface). <br/>
Many helpful tips and techniques from [28 Jupyter Notebook Tips, Tricks, and Shortcuts](https://www.dataquest.io/blog/jupyter-notebook-tips-tricks-shortcuts/).

## Setup

### Create a Kaggle Account

#### 1. Register for an account

In order to download Kaggle competition data you will first need to create a [Kaggle](https://www.kaggle.com/) account.

#### 2. Create an API key

Once you have registered for a Kaggle account you will need to create [API credentials](https://github.com/Kaggle/kaggle-api#api-credentials) in order to be able to use the `kaggle` CLI to download data.

* Go to the `Account` tab of your user profile, 
* and click `Create New API Token` from the API section.  

This generates a `kaggle.json` file (with 'username' and 'key' values) to download.


### Setup Colab

In order to run this notebook in [Google Colab](https://colab.research.google.com) you will need a [Google Account](https://accounts.google.com/).  Sign-in to your Google account, if necessary, and then start the notebook.

Change Google Colab runtime to use GPU:

* Click `Runtime` -> `Change runtime type` menu item
* Specify `Runtime type` as `Python 3`
* Specify `Hardware accelerator` as `GPU`
* Click **[Save]** button

The session indicator (toolbar / status ribbon under menu) should briefly appear as `Connecting...`.  When the session restarts, continue with the next cell (specifying TensorFlow version v2.x):

In [ ]:
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

### Download Data

In [ ]:
import pathlib
import tensorflow.keras.utils as Kutils

def cache_mnist_data():
    for n in ["mnist.npz", "kaggle/train.csv", "kaggle/test.csv"]:
        path = pathlib.Path("../datasets/mnist/%s" % n).absolute()
        DATA_URL = "file:///" + str(path)
        data_file_path = Kutils.get_file(n.replace('/','-mnist-'), DATA_URL)
        print("cached file: %s" % n)

def cache_cifar10_data():
    for n in ["cifar-10.npz", "cifar-10-batches-py.tar.gz"]:
        path = pathlib.Path("../datasets/cifar10/%s" % n).absolute()
        DATA_URL = "file:///" + str(path)
        if path.is_file(): 
            data_file_path = Kutils.get_file(n, DATA_URL)
            print("cached file: %s" % n)
        else:
            print("FAILED: First fetch file: %s" % n)

def cache_models():
    for n in ["vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"]:
        path = pathlib.Path("../models/%s" % n).absolute()
        DATA_URL = "file:///" + str(path)
        if path.is_file(): 
            data_file_path = Kutils.get_file(n, DATA_URL, cache_subdir='models')
            print("cached file: %s" % n)

#### Download MNIST Data

If you are using Binder to run this notebook, then the data is already downloaded and available.  Skip to the next step.

If you are using Google Colab to run this notebook, then you will need to download the data before proceeding.

##### Download MNIST from Kaggle

**Note:** Before attempting to download the competition data you will need to login to your [Kaggle](https://www.kaggle.com) account and accept the rules for this competition.

Set your Kaggle username and API key (from the `kaggle.json` file) into the cell below, and execute the code to download the Kaggle [Digit Recognizer: Learn computer vision with the famous MNIST data](https://www.kaggle.com/c/digit-recognizer) competition data. 

In [ ]:
%%bash
# NOTE: Replace YOUR_USERNAME and YOUR_API_KEY with actual credentials 
export KAGGLE_USERNAME="YOUR_USERNAME"
export KAGGLE_KEY="YOUR_API_KEY"
kaggle competitions download -c digit-recognizer -p ../datasets/mnist/kaggle

In [ ]:
%%bash
unzip ../datasets/mnist/kaggle/digit-recognizer.zip -d ../datasets/mnist/kaggle

##### (Alternative) Download MNIST from GitHub

If you are running this notebook using Google Colab, but did *not* create a Kaggle account and API key, then  dowload the data from our GitHub repository by running the code in the following cells.

In [ ]:
import pathlib
import requests

def fetch_mnist_data():
    RAW_URL = "https://github.com/holstgr-kaust/keras-tutorials/raw/master/datasets/mnist"
    DEST_DIR = pathlib.Path('../datasets/mnist')

    DEST_DIR.mkdir(parents=True, exist_ok=True)
    for n in ["mnist.npz", "kaggle/train.csv", "kaggle/test.csv", "kaggle/sample_submission.csv"]:
        path = DEST_DIR / n
        if not path.is_file():  # Don't download if file exists
            with path.open(mode = 'wb') as f:
                response = requests.get(RAW_URL + "/" + n)
                f.write(response.content)

In [ ]:
fetch_mnist_data()
cache_mnist_data()

##### (Alternative) Download MNIST with Keras

If you are running this notebook using Google Colab, but did *not* create a Kaggle account and API key, then dowload the data using the Keras load_data() API by running the code in the following cells.

In [ ]:
from tensorflow.keras.datasets import mnist
cache_mnist_data()
mnist.load_data();

#### Download CIFAR10 Data

If you are using Binder to run this notebook, then the data is already downloaded and available.  Skip to the next step.

If you are using Google Colab to run this notebook, then you will need to download the data before proceeding.

##### Download CIFAR10 from Kaggle

**Note:** Before attempting to download the competition data you will need to login to your [Kaggle](https://www.kaggle.com) account.

Set your Kaggle username and API key (from the `kaggle.json` file) into the cell below, and execute the code to download the Kaggle [Digit Recognizer: Learn computer vision with the famous MNIST data](https://www.kaggle.com/c/digit-recognizer) competition data. 

In [ ]:
%%bash
# NOTE: Replace YOUR_USERNAME and YOUR_API_KEY with actual credentials 
export KAGGLE_USERNAME="YOUR_USERNAME"
export KAGGLE_KEY="YOUR_API_KEY"
kaggle datasets download guesejustin/cifar10-keras-files-cifar10load-data -p ../datasets/cifar10/

In [ ]:
%%bash
unzip ../datasets/cifar10/cifar10-keras-files-cifar10load-data.zip -d ../datasets/cifar10

##### (Alternative) Download CIFAR10 from GitHub

If you are running this notebook using Google Colab, but did *not* create a Kaggle account and API key, then  dowload the data from our GitHub repository by running the code in the following cells.

In [ ]:
import os
import pathlib
import requests

def fetch_cifar10_data():
    RAW_URL = "https://github.com/holstgr-kaust/keras-tutorials/raw/master/datasets/cifar10"
    DEST_DIR = pathlib.Path('../datasets/cifar10')

    DEST_DIR.mkdir(parents=True, exist_ok=True)
    for n in ["cifar-10.npz", "cifar-10-batches-py.tar.gz"]:
        path = DEST_DIR / n
        if not path.is_file():  # Don't download if file exists
            with path.open(mode = 'wb') as f:
                response = requests.get(RAW_URL + "/" + n)
                f.write(response.content)
            print("downloaded file: %s" % n)

In [ ]:
fetch_cifar10_data()
cache_cifar10_data()

In [ ]:
%%bash
DEST_DIR='../datasets/cifar10'
tar xvpf "${DEST_DIR}/cifar-10-batches-py.tar.gz" --directory="${DEST_DIR}" 

##### (Alternative) Download CIFAR10 with Keras

If you are running this notebook using Google Colab, but did *not* create a Kaggle account and API key, then dowload the data using the Keras load_data() API by running the code in the following cells.

In [ ]:
from tensorflow.keras.datasets import cifar10
cache_cifar10_data()
cifar10.load_data();

## Tutorial

### Setup

Initialize the Python environment by importing and verifying the modules we will use.

In [ ]:
import os
import sys
import pathlib
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras

`%matplotlib inline` is a magic command that makes *matplotlib* charts and plots appear was outputs in the notebook.

`%matplotlib notebook` enables semi-interactive plots that can be enlarged, zoomed, and cropped while the plot is active.  One issue with this option is that new plots appear in the active plot widget, not in the cell where the data was produced.

In [ ]:
%matplotlib inline

In [ ]:
# Verify runtime environment

try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
    IS_COLAB = True
except Exception:
    IS_COLAB = False
print("is_colab:", IS_COLAB)

assert tf.__version__ >= "2.0", "TensorFlow version >= 2.0 required."
print("tensorflow_version:", tf.__version__)

assert sys.version_info >= (3, 5), "Python >= 3.5 required."
print("python_version:", "%s.%s.%s-%s" % (sys.version_info.major, 
                                          sys.version_info.minor,
                                          sys.version_info.micro,
                                          sys.version_info.releaselevel
                                         ))

print("executing_eagerly:", tf.executing_eagerly())

__physical_devices = tf.config.experimental.list_physical_devices('GPU')
if len(__physical_devices) == 0:
    print("No GPUs available. Expect training to be very slow.")
    if IS_COLAB:
        print("Go to `Runtime` > `Change runtime` and select a GPU hardware accelerator."
              "Then `Save` to restart session.")
else:
    print("is_built_with_cuda:", tf.test.is_built_with_cuda())
    print("is_gpu_available:", tf.test.is_gpu_available(), [d.name for d in __physical_devices])

### CIFAR10 - Dataset Processing

The previously acquired CIFAR10 dataset is the essential input needed to train an image classification model. Before using the dataset, there are several preprocessing steps required to load the data, and create the correctly sized training, validation, and testing arrays used as input to the network.

The following data preparation steps are needed before they can become inputs to the network:

* Cache the downloaded dataset (to use Keras `load_data()` functionality).
* Load the dataset (CIFAR10 is small, and fits into a `numpy` array).
* Verify the shape and type of the data, and understand it...
* Convert label indices into categorical vectors.
* Convert image data from integer to float values, and normalize.
  * Verify converted input data.

#### Cache Data

Make downloaded data available to Keras.  Provide dataset utility functions.

In [ ]:
# Cache CIFAR10 Datasets
cache_cifar10_data()

In [ ]:
%%bash
find ~/.keras -name "cifar-10*" -type f

In [ ]:
# Helper functionality to provide human-readable labels
cifar10_label_names = ['airplane', 'automobile', 
                       'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 
                       'ship', 'truck']

def cifar10_index_label(idx):
    return cifar10_label_names[int(idx)]

def cifar10_category_label(cat):
    return cifar10_index_label(cat.argmax())

def cifar10_label(v):
    return cifar10_index_label(v) if np.isscalar(v) or np.size(v) == 1 else cifar10_category_label(v)

#### Load Data

In [ ]:
from tensorflow.keras.datasets import cifar10

# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

**Note:** Backup plan: Only run the following cell if the data didn't load via `cifar10.load_data` above.

In [ ]:
# Try secondary data source if the first didn't work
try:
    print("data loaded." if type((x_train, y_train, x_test, y_test)) else "load failed...")
except NameError:
    with np.load('../datasets/cifar10/cifar-10.npz') as data:
        x_train = data['x_train']
        y_train = data['y_train']
        x_test = data['x_test']
        y_test = data['y_test']
    print("alternate data load." if type((x_train, y_train, x_test, y_test)) else "failed...")

#### Explore Data

Explore data types, shape, and value ranges.  Ensure they make sense, and you understand the data well.

In [ ]:
print('x_train type:', type(x_train), ',', 'y_train type:', type(y_train))
print('x_train dtype:', x_train.dtype, ',', 'y_train dtype:', y_train.dtype)
print('x_train shape:', x_train.shape, ',', 'y_train shape:', y_train.shape)
print('x_test shape:', x_test.shape, ',', 'y_test shape:', y_test.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

In [ ]:
print('x_train (min, max, mean): (%s, %s, %s)' % (x_train.min(), x_train.max(), x_train.mean()))
print('y_train (min, max): (%s, %s)' % (y_train.min(), y_train.max()))

In [ ]:
def cifar10_imageset_plot(img_data=None):
    (x_imgs, y_imgs) = img_data if img_data else (x_train, y_train)
    fig = plt.figure(figsize=(16,8))

    for i in range(40):
        plt.subplot(4, 10, i + 1)
        plt.xticks([])
        plt.yticks([])
        idx = int(random.uniform(0, x_imgs.shape[0]))
        plt.title(cifar10_label(y_imgs[idx]))
        plt.imshow(x_imgs[idx], cmap=plt.get_cmap('gray'))
    plt.show()

In [ ]:
# Show array of random labelled images with matplotlib (re-run cell to see new examples)
cifar10_imageset_plot((x_train, y_train))

In [ ]:
def histogram_plot(img_data=None):
    (x_data, y_data) = img_data if img_data else (x_train, y_train)
    
    hist, bins = np.histogram(y_data, bins = range(int(y_data.min()), int(y_data.max() + 2)))

    fig = plt.figure(figsize=(12,5))

    plt.subplot(1,2,1)
    plt.hist(y_data, bins = range(int(y_data.min()), int(y_data.max() + 2)))
    plt.xticks(range(int(y_data.min()), int(y_data.max() + 2)))
    plt.title("y histogram")
    plt.subplot(1,2,2)
    plt.hist(x_data.flat, bins = range(int(x_data.min()), int(x_data.max() + 2)))
    plt.title("x histogram")
    plt.tight_layout()
    plt.show()

    print('y histogram counts:', hist)

In [ ]:
histogram_plot((x_train, y_train))

In [ ]:
histogram_plot((x_test, y_test))

The data looks reasonable: there are sufficient examples for each category (y_train) and a near-normal distribution of pixel values that appears similar in both the train and test datasets.

##### Visualizing training samples using PCA

[Principal Components Analysis (PCA)](https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) can be used as a visualization tool to see if there are any obvious patterns in the training samples.

In [ ]:
import sklearn
import sklearn.decomposition

_prng = np.random.RandomState(42)

pca = sklearn.decomposition.PCA(n_components=3, random_state=_prng)

x_train_flat = x_train.reshape(*x_train.shape[:1], -1)
y_train_flat = y_train.reshape(y_train.shape[0])
print("x_train:", x_train.shape, "y_train", y_train.shape)
print("x_train_flat:", x_train_flat.shape, "y_train_flat", y_train_flat.shape)
pca_train_features = pca.fit_transform(x_train_flat, y_train_flat)
print("pca_train_features:", pca_train_features.shape)

# Sample 10% of the PCA results
_idxs = _prng.randint(y_train_flat.shape[0], size=y_train_flat.shape[0] // 10)
pca_features = pca_train_features[_idxs]
pca_category = y_train_flat[_idxs]
print("pca_features:", pca_features.shape, 
      "pca_category", pca_category.shape, 
      "min,max category:", pca_category.min(), pca_category.max())

In [ ]:
def category_scatter_plot(features, category):
    num_category = 1 + category.max() - category.min()

    fig, ax = plt.subplots(1, 1, figsize=(12, 10))
    cm = plt.cm.get_cmap('tab10', num_category)
    sc = ax.scatter(features[:,0], features[:,1], c=category, alpha=0.4, cmap=cm)
    ax.set_xlabel("Component 1")
    ax.set_ylabel("Component 2")
    ax.set_title("CIFAR10 - PCA")
    plt.colorbar(sc)
    plt.show()

In [ ]:
from mpl_toolkits.mplot3d import Axes3D

def category_scatter3d_plot(features, category):
    num_category = 1 + category.max() - category.min()
    mean_feat = np.mean(features, axis=0)
    std_feat = np.std(features, axis=0)
    min_range = mean_feat - std_feat
    max_range = mean_feat + std_feat
    
    fig = plt.figure(figsize=(12, 10))
    cm = plt.cm.get_cmap('tab10', num_category)
    ax = fig.add_subplot(111, projection='3d')
    sc = ax.scatter(features[:,0], features[:,1], features[:,2],
                    c=category, alpha=0.85, cmap=cm)
    ax.set_xlabel("Component 1")
    ax.set_ylabel("Component 2")
    ax.set_zlabel("Component 3")
    ax.set_title("CIFAR10 - PCA")
    ax.set_xlim(2.0 * min_range[0], 2.0 * max_range[0])
    ax.set_ylim(2.0 * min_range[1], 2.0 * max_range[1])
    ax.set_zlim(2.0 * min_range[2], 2.0 * max_range[2])
    plt.colorbar(sc)
    plt.show()

In [ ]:
category_scatter_plot(pca_features, pca_category)

**Note:** 3D PCA plot works best with `%matplotlib notebook` to enable interactive rotation.

In [ ]:
category_scatter3d_plot(pca_features, pca_category)

The data in its original image space does not appear to cluster into corresponding categories.

##### Visualizing training sample using t-SNE

[t-distributed Stochastic Neighbor Embedding (t-SNE)](https://scikit-learn.org/stable/modules/generated/sklearn.manifold.TSNE.html#sklearn.manifold.TSNE) is a tool to visualize high-dimensional data. It converts similarities between data points to joint probabilities and tries to minimize the Kullback-Leibler divergence between the joint probabilities of the low-dimensional embedding and the high-dimensional data. For more details on t-SNE including other use cases see this excellent *Toward Data Science* [blog post](https://towardsdatascience.com/an-introduction-to-t-sne-with-python-example-5a3a293108d1)

It is highly recommended to use another dimensionality reduction method (e.g. PCA) to reduce the number of dimensions to a reasonable amount if the number of features is very high. This will suppress some noise and speed up the computation of pairwise distances between samples.

In [ ]:
import sklearn
import sklearn.decomposition
import sklearn.pipeline
import sklearn.manifold

_prng = np.random.RandomState(42)

embedding2_pipeline = sklearn.pipeline.make_pipeline(
    sklearn.decomposition.PCA(n_components=0.95, random_state=_prng),
    sklearn.manifold.TSNE(n_components=2, random_state=_prng))

embedding3_pipeline = sklearn.pipeline.make_pipeline(
    sklearn.decomposition.PCA(n_components=0.95, random_state=_prng),
    sklearn.manifold.TSNE(n_components=3, random_state=_prng))

In [ ]:
# Sample 10% of the data

_prng = np.random.RandomState(42)

_idxs = _prng.randint(y_train_flat.shape[0], size=y_train_flat.shape[0] // 10)
tsne_features = x_train_flat[_idxs]
tsne_category = y_train_flat[_idxs]
print("tsne_features:", tsne_features.shape, 
      "tsne_category", tsne_category.shape, 
      "min,max category:", tsne_category.min(), tsne_category.max())

In [ ]:
# t-SNE is SLOW (but can be GPU accelerated!); 
#       lengthy operation, be prepared to wait...

transform2_tsne_features = embedding2_pipeline.fit_transform(tsne_features)

print("transform2_tsne_features:", transform2_tsne_features.shape)
for i in range(2):
    print("min,max features[%s]:" % i, 
          transform2_tsne_features[:,i].min(), 
          transform2_tsne_features[:,i].max())

In [ ]:
category_scatter_plot(transform2_tsne_features, tsne_category)

In [ ]:
# t-SNE is SLOW (but can be GPU accelerated!); 
#       lengthy operation, be prepared to wait...

transform3_tsne_features = embedding3_pipeline.fit_transform(tsne_features)

print("transform3_tsne_features:", transform3_tsne_features.shape)
for i in range(3):
    print("min,max features[%s]:" % i, 
          transform3_tsne_features[:,i].min(), 
          transform3_tsne_features[:,i].max())

In [ ]:
category_scatter3d_plot(transform3_tsne_features, tsne_category)

t-SNE relates the data points (images) according to their closest neighbours.  Hints of underlying categories appear; but not cleanly seperable into the original categories.

#### Data Conversion

The data type for the training data is `uint8`, while the input type for the network will be `float32` so the data must be converted.  Also, the data should be normalized, and the labels need to be categorical.  I.e., instead of label existing as 10 different values in a 1-D space, they need to exist as Boolean values in a 10-D space — one dimension for each category, and either a 0 or 1 value in each dimension to represent membership in that category.

* https://keras.io/examples/cifar10_cnn/

In [ ]:
num_classes = (y_train.max() - y_train.min()) + 1
print('num_classes =', num_classes)

y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [ ]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

train_data = (x_train, y_train)
test_data = (x_test, y_test)

In [ ]:
print('x_train type:', type(x_train))
print('x_train dtype:', x_train.dtype)
print('x_train shape:', x_train.shape)
print('x_test shape:', x_test.shape)

print('y_train type:', type(y_train))
print('y_train dtype:', y_train.dtype)
print('y_train shape:', y_train.shape)
print('y_test shape:', y_test.shape)

## Acquire Pre-Trained Network

Download an *ImageNet* pretrained VGG16 network[<sup>1</sup>](#fn1), sans classification layer, shaped for 32x32px colour images<sup>[*](https://github.com/fchollet/deep-learning-models/releases/download/v0.1/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5)</sup> (the smallest supported size).  This image-feature detection network is an example of a deep CNN (Convolutional Neural Network).

**Note:** The network must be fixed – it was already trained on a very large dataset, so training it on our smaller dataset would result in it un-learning valuable generic features.

<span id="fn1"><sup>[1]</sup> *Very Deep Convolutional Networks for Large-Scale Image Recognition** by Karen Simonyan and Andrew Zisserman, [arXiv (2014)](https://arxiv.org/abs/1409.1556).</span>

In [ ]:
cache_models()

In [ ]:
from tensorflow.keras.applications import VGG16

conv_base = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
conv_base.trainable = False
conv_base.summary()

The input layer shape and data type should match with the input data:

*Note:* The first dimension of the shape will differ; the input layer has `None` to indicate it accepts an a batch sized collection of arrays of the remaining shape.  The data shape has a number indicating how many samples it contains.

In [ ]:
print("input layer shape:", conv_base.layers[0].input.shape)
print("input layer dtype:", conv_base.layers[0].input.dtype)
conv_base.layers[0].input

In [ ]:
print("input data shape:", x_train.shape)
print("input data dtype:", x_train.dtype)

### Explore Convolutional Layers

In [ ]:
def cifar10_image_plot(img_data=None, image_index=None):
    (x_imgs, y_imgs) = img_data if img_data else (x_train, y_train)

    if not image_index:
        image_index = int(random.uniform(0, x_imgs.shape[0]))

    plt.imshow(x_imgs[image_index], cmap='gray')
    plt.title("%s" % cifar10_label(y_imgs[image_index]))
    plt.xlabel("#%s" % image_index)
    plt.show()
    
    return image_index

def get_model_layer(model, layer_name):
    if type(layer_name) == str:
        layer = model.get_layer(layer_name)
    else:
        m = model
        for ln in layer_name:
            model = m
            m = m.get_layer(ln)
        layer = m
    return (model, layer)

In [ ]:
def visualize_conv_layer_weights(model, layer_name):
    (model, layer) = get_model_layer(model, layer_name)
    layer_weights = layer.weights[0]

    max_size = layer_weights.shape[3]
    col_size = 12
    row_size = int(np.ceil(float(max_size) / float(col_size)))

    print("conv layer: %s shape: %s size: (%s,%s) count: %s" % 
          (layer_name,
           layer_weights.shape,
           layer_weights.shape[0], layer_weights.shape[1],
           max_size))

    fig, ax = plt.subplots(row_size, col_size, figsize=(12, 1.2 * row_size))
    idx = 0

    for row in range(0,row_size):
        for col in range(0,col_size):
            ax[row][col].set_xticks([])
            ax[row][col].set_yticks([])
            if idx < max_size:
                ax[row][col].imshow(layer_weights[:, :, 0, idx], cmap='gray')
            else:
                fig.delaxes(ax[row][col])
            idx += 1

    plt.tight_layout()
    plt.show()

In [ ]:
def visualize_conv_layer_output(model, layer_name, image_index=None):
    (model, layer) = get_model_layer(model, layer_name)
    layer_output = layer.output

    if not image_index:
        image_index = cifar10_image_plot()
        
    intermediate_model = keras.models.Model(inputs = model.input, outputs=layer_output) 
    intermediate_prediction = intermediate_model.predict(x_train[image_index].reshape(1,32,32,3))
  
    max_size = layer_output.shape[3]
    col_size = 10
    row_size = int(np.ceil(float(max_size) / float(col_size)))

    print("conv layer: %s shape: %s size: (%s,%s) count: %s" % 
          (layer_name,
           layer_output.shape,
           layer_output.shape[1], layer_output.shape[2],
           max_size))
    
    fig, ax = plt.subplots(row_size, col_size, figsize=(12, 1.2 * row_size))
    idx = 0

    for row in range(0,row_size):
        for col in range(0,col_size):
            ax[row][col].set_xticks([])
            ax[row][col].set_yticks([])
            if idx < max_size:
                ax[row][col].imshow(intermediate_prediction[0, :, :, idx], cmap='gray')
            else:
                fig.delaxes(ax[row][col])
            idx += 1

    plt.tight_layout()
    plt.show()

In [ ]:
from tensorflow.keras import backend as K

def process_image(x):
    epsilon = 1e-5
    # Normalizes the tensor: centers on 0, ensures that std is 0.1 Clips to [0, 1]
    x -= x.mean()
    x /= (x.std() + epsilon)
    x *= 0.1
    x += 0.5
    x = np.clip(x, 0, 1)
    x *= 255
    x = np.clip(x, 0, 255).astype('uint8')
    return x

def generate_response_pattern(model, conv_layer_output, filter_index=0):
    #step_size = 1.0
    epsilon = 1e-5

    img_tensor = tf.Variable(tf.random.uniform((1, 32, 32, 3)) * 20 + 128.0, trainable=True)

    response_model = keras.models.Model([model.inputs], [conv_layer_output])

    for i in range(40):
        with tf.GradientTape() as gtape:
            layer_output = response_model(img_tensor)
            loss = K.mean(layer_output[0, :, :, filter_index])
            grads = gtape.gradient(loss, img_tensor)
            grads /= (K.sqrt(K.mean(K.square(grads))) + epsilon)
        img_tensor = tf.Variable(tf.add(img_tensor, grads))

    img = np.array(img_tensor[0])
    return process_image(img)

In [ ]:
def visualize_conv_layer_response(model, layer_name):
    (model, layer) = get_model_layer(model, layer_name)
    layer_output = layer.output
    
    max_size = layer_output.shape[3]
    col_size = 12
    row_size = int(np.ceil(float(max_size) / float(col_size)))

    print("conv layer: %s shape: %s size: (%s,%s) count: %s" % 
          (layer_name,
           layer_output.shape,
           layer_output.shape[1], layer_output.shape[2],
           max_size))
    
    fig, ax = plt.subplots(row_size, col_size, figsize=(12, 1.2 * row_size))
    idx = 0

    for row in range(0,row_size):
        for col in range(0,col_size):
            ax[row][col].set_xticks([])
            ax[row][col].set_yticks([])
            if idx < max_size:
                img = generate_response_pattern(model, layer_output, idx)
                ax[row][col].imshow(img, cmap='gray')
                ax[row][col].set_title("%s" % idx)
            else:
                fig.delaxes(ax[row][col])
            idx += 1

    plt.tight_layout()
    plt.show()

In [ ]:
for n in [l.name for l in conv_base.layers if isinstance(l, keras.layers.Conv2D)][:4]:
    visualize_conv_layer_weights(conv_base, n)

In [ ]:
image_index = cifar10_image_plot()
for n in [l.name for l in conv_base.layers if isinstance(l, keras.layers.Conv2D)][:7]:
    visualize_conv_layer_output(conv_base, n, image_index)

In [ ]:
for n in [l.name for l in conv_base.layers if isinstance(l, keras.layers.Conv2D)][:4]:
    visualize_conv_layer_response(conv_base, n)

In [ ]:
# NOTE: Visualize mid to higher level convolutional layers; 
#       lengthy operation, be prepared to wait...
for n in [l.name for l in conv_base.layers if isinstance(l, keras.layers.Conv2D)][4:]:
    visualize_conv_layer_response(conv_base, n)

### CNN Base + Classifier Model

Create a simple model that has the pre-trained CNN (Convolutional Neural Network) as a base, and adds a basic classifier on top.

Notice the split of total parameters (\~15 million) between trainable (\~0.3 million for our classifier) and non-trainable (\~14.7 million for the pre-trained CNN).

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Activation, Dropout
from tensorflow.keras.applications import VGG16

def create_cnnbase_classifier_model(conv_base=None):
    if not conv_base:
        conv_base = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
        conv_base.trainable = False

    model = Sequential()
    model.add(conv_base)
    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

    return model

In [ ]:
model_transfer_cnn = create_cnnbase_classifier_model(conv_base)
model_transfer_cnn.summary()

### Train Model

In [ ]:
batch_size = 128 #32
epochs = 25 #100
learning_rate = 1e-3 #1e-4
decay = 1e-6

In [ ]:
from tensorflow.keras.optimizers import RMSprop

model_transfer_cnn.compile(loss='categorical_crossentropy',
                           optimizer=RMSprop(learning_rate=learning_rate, decay=decay),
                           metrics=['accuracy'])

In [ ]:
history = model_transfer_cnn.fit(x_train, y_train,
                                 batch_size=batch_size,
                                 epochs=epochs,
                                 validation_data=(x_test, y_test),
                                 shuffle=True)

### Evaluate Model

Visualize accuracy and loss for training and validation.

* https://keras.io/visualization/

In [ ]:
def history_plot(history):
    fig = plt.figure(figsize=(12,5))

    plt.title('Model accuracy & loss')

    # Plot training & validation accuracy values
    ax1 = fig.add_subplot()
    #ax1.set_ylim(0, 1.1 * max(history.history['loss']+history.history['val_loss']))
    ax1.set_prop_cycle(color=['green', 'red'])
    p1 = ax1.plot(history.history['loss'], label='Train Loss')
    p2 = ax1.plot(history.history['val_loss'], label='Test Loss')

    # Plot training & validation loss values
    ax2 = ax1.twinx()
    ax2.set_ylim(0, 1.1 * max(history.history['accuracy']+history.history['val_accuracy']))
    ax2.set_prop_cycle(color=['blue', 'orange'])
    p3 = ax2.plot(history.history['accuracy'], label='Train Acc')
    p4 = ax2.plot(history.history['val_accuracy'], label='Test Acc')

    ax1.set_ylabel('Loss')
    ax1.set_xlabel('Epoch')
    ax2.set_ylabel('Accuracy')

    pz = p3 + p4 + p1 + p2
    plt.legend(pz, [l.get_label() for l in pz], loc='center right')
    plt.show()

In [ ]:
history_plot(history)

In [ ]:
# Score trained model.
scores = model_transfer_cnn.evaluate(x_test, y_test, verbose=0)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

In [ ]:
def prediction_plot(model, test_data):
    (x_test, y_test) = test_data
    fig = plt.figure(figsize=(16,8))
    correct = 0
    total = 0
    
    for i in range(40):
        plt.subplot(4, 10, i + 1)
        plt.xticks([])
        plt.yticks([])
        idx = int(random.uniform(0, x_test.shape[0]))
        result = model.predict(x_test[idx:idx+1])[0]
        rCorrect = True if cifar10_label(y_test[idx]) == cifar10_label(result) else False
        rSym = '✔' if rCorrect else '✘'
        correct += 1 if rCorrect else 0
        total += 1
        plt.title("%s %s" % (rSym, cifar10_label(result)))
        plt.imshow(x_test[idx], cmap=plt.get_cmap('gray'))
    plt.show()
    
    print("% 3.2f%% correct (%s/%s)" % (100.0 * float(correct) / float(total), correct, total))

In [ ]:
def prediction_classes_plot(model, test_data):
    (x_test, y_test) = test_data
    fig = plt.figure(figsize=(16,8))
    correct = 0
    total = 0
    
    for i in range(40):
        plt.subplot(4, 10, i + 1)
        plt.xticks([])
        plt.yticks([])
        idx = int(random.uniform(0, x_test.shape[0]))
        result = model.predict_classes(x_test[idx:idx+1])[0]
        rCorrect = True if cifar10_label(y_test[idx]) == cifar10_label(result) else False
        rSym = '✔' if rCorrect else '✘'
        correct += 1 if rCorrect else 0
        total += 1
        plt.title("%s %s" % (rSym, cifar10_label(result)))
        plt.imshow(x_test[idx], cmap=plt.get_cmap('gray'))
    plt.show()
    
    print("% 3.2f%% correct (%s/%s)" % (100.0 * float(correct) / float(total), correct, total))

In [ ]:
def prediction_proba_plot(model, test_data):
    (x_test, y_test) = test_data
    fig = plt.figure(figsize=(15,15))
    
    for i in range(10):
        plt.subplot(10, 2, (2*i) + 1)
        plt.xticks([])
        plt.yticks([])
        idx = int(random.uniform(0, x_test.shape[0]))
        result = model.predict_proba(x_test[idx:idx+1])[0] * 100 # prob -> percent
        plt.title("%s" % cifar10_label(y_test[idx]))
        plt.xlabel("#%s" % idx)
        plt.imshow(x_test[idx], cmap=plt.get_cmap('gray'))
        
        ax = plt.subplot(10, 2, (2*i) + 2)
        plt.bar(np.arange(len(result)), result, label='%')
        plt.xticks(range(0, len(result) + 1))
        ax.set_xticklabels(cifar10_label_names)
        plt.title("classifier probabilities")

        plt.tight_layout()
    plt.show()

* *Grad-CAM: Visual Explanations from Deep Networks via Gradient-based Localization* by Ramprasaath Selvaraju, Michael Cogswell, Abhishek Das, Ramakrishna Vedantam, Devi Parikh, and Dhruv Batra [arXiv (2016)](https://arxiv.org/abs/1610.02391)
* https://jacobgil.github.io/deeplearning/class-activation-maps

In [ ]:
from tensorflow.keras import backend as K

def generate_activation_pattern(model, conv_layer_output, category_idx, image):
    epsilon = 1e-10

    activation_model = keras.models.Model([model.inputs], [conv_layer_output, model.output])

    with tf.GradientTape() as gtape:
        conv_output, prediction = activation_model(image)
        category_output = prediction[:, category_idx]
        grads = gtape.gradient(category_output, conv_output)
        pooled_grads = K.mean(grads, axis=(0, 1, 2))

    heatmap = tf.reduce_mean(tf.multiply(pooled_grads, conv_output), axis=-1) * -1.
    heatmap = np.maximum(heatmap, 0)
    heatmap /= np.max(heatmap) + epsilon
    return(heatmap)

In [ ]:
def activation_plot(model, layer_name, image_data, image_index=None):
    (layer_model, conv_layer) = get_model_layer(model, layer_name)
    (x_imgs, y_cat) = image_data

    if not image_index:
        image_index = int(random.uniform(0, x_imgs.shape[0]))
    
    image = x_imgs[image_index:image_index+1]

    fig = plt.figure(figsize=(16,8))

    plt.subplot(1, num_classes + 2, 1)
    plt.xticks([])
    plt.yticks([])
    plt.title(cifar10_label(y_cat[image_index]))
    plt.xlabel("#%s" % image_index)
    plt.imshow(image.reshape(32, 32, 3))

    result = model.predict(image)[0]

    for i in range(num_classes):
        activation = generate_activation_pattern(model, conv_layer.output, i, image)
        activation = np.copy(activation)
        plt.subplot(1, num_classes + 2, i + 2)
        plt.xticks([])
        plt.yticks([])
        plt.title(cifar10_label(i))
        plt.xlabel("(% 3.2f%%)" % (result[i] * 100.0))
        plt.imshow(activation[0])
        
    plt.show()

In [ ]:
prediction_classes_plot(model_transfer_cnn, (x_test, y_test))

In [ ]:
prediction_proba_plot(model_transfer_cnn, (x_test, y_test))

In [ ]:
# TODO: Complete activation plot
#activation_plot(model_transfer_cnn, ('vgg16', 'block5_conv3'), (x_test, y_test), 1)

### CNN Classifier Model

Create a basic CNN (Convolutional Neural Network) based classifier from scratch.

Notice the total number of parameters (\~1.25 million) in this smaller network.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Activation, Dropout, Conv2D, MaxPooling2D

def create_cnn_classifier_model():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), padding='same',
                     input_shape=x_train.shape[1:]))
    model.add(Activation('relu'))
    model.add(Conv2D(32, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Conv2D(64, (3, 3), padding='same'))
    model.add(Activation('relu'))
    model.add(Conv2D(64, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.add(Dense(512))
    model.add(Activation('relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes))
    model.add(Activation('softmax'))

    return model

In [ ]:
model_simple_cnn = create_cnn_classifier_model()
model_simple_cnn.summary()

In [ ]:
batch_size = 128 #32
epochs = 25 #100
learning_rate = 1e-3 #1e-4
decay = 1e-6

In [ ]:
from tensorflow.keras.optimizers import RMSprop

model_simple_cnn.compile(loss='categorical_crossentropy',
                         optimizer=RMSprop(learning_rate=learning_rate, decay=decay),
                         metrics=['accuracy'])

In [ ]:
history = model_simple_cnn.fit(x_train, y_train,
                               batch_size=batch_size,
                               epochs=epochs,
                               validation_data=(x_test, y_test),
                               shuffle=True)

In [ ]:
history_plot(history)

In [ ]:
# Score trained model.
scores = model_simple_cnn.evaluate(x_test, y_test, verbose=0)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

In [ ]:
prediction_classes_plot(model_simple_cnn, (x_test, y_test))

In [ ]:
prediction_proba_plot(model_simple_cnn, (x_test, y_test))

In [ ]:
for n in [l.name for l in model_simple_cnn.layers if isinstance(l, keras.layers.Conv2D)][:4]:
    visualize_conv_layer_weights(model_simple_cnn, n)

In [ ]:
image_index = cifar10_image_plot()
for n in [l.name for l in model_simple_cnn.layers if isinstance(l, keras.layers.Conv2D)]:
    visualize_conv_layer_output(model_simple_cnn, n, image_index)

In [ ]:
for n in [l.name for l in model_simple_cnn.layers if isinstance(l, keras.layers.Conv2D)][:4]:
    visualize_conv_layer_response(model_simple_cnn, n)

In [ ]:
n = [l.name for l in model_simple_cnn.layers if isinstance(l, keras.layers.Conv2D)][-1]
print(n)
for i in range(5):
    activation_plot(model_simple_cnn, n, (x_test, y_test))

### Combined Models

Keras supports a functional interface to take network architectures beyond simply sequential networks.

We'll demonstrate by creating a new network which combines to two CNN classifier networks we created previously.

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Concatenate, Flatten, Dense, Activation, Dropout
from tensorflow.keras.optimizers import RMSprop

def create_combined_classifier_model(trained_model1=None, trained_model2=None):
    if trained_model1:
        network1 = trained_model1
        network1.trainable = False
    else:
        network1 = create_cnnbase_classifier_model()

    if trained_model2:
        network2 = trained_model2
        network2.trainable = False
    else:
        network2 = create_cnn_classifier_model()

    inputs = Input(shape=(32,32,3), name='cifar10_image')
    c1 = network1(inputs)
    c2 = network2(inputs)
    c = Concatenate()([c1, c2])
    x = Dense(512)(c)
    x = Activation('relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(num_classes)(x)
    outputs = Activation('softmax')(x)
    
    model = Model(inputs=inputs, outputs=outputs, name='combined_cnn_classifier')
    return model

#### Combining Pre-Trained Models

This version of the combined classifier uses both of the trained networks we created previously.

In [ ]:
model_combined = create_combined_classifier_model(model_transfer_cnn, model_simple_cnn)
model_combined.summary()

In [ ]:
keras.utils.plot_model(model_combined)

In [ ]:
batch_size = 128 #32
epochs = 5 #100
learning_rate = 1e-3 #1e-4
decay = 1e-6

model_combined.compile(loss='categorical_crossentropy',
                         optimizer=RMSprop(learning_rate=learning_rate, decay=decay),
                         metrics=['accuracy'])

In [ ]:
history = model_combined.fit(x_train, y_train,
                             batch_size=batch_size,
                             epochs=epochs,
                             validation_data=(x_test, y_test),
                             shuffle=True)

In [ ]:
history_plot(history)

In [ ]:
# Score trained model.
scores = model_combined.evaluate(x_test, y_test, verbose=0)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

In [ ]:
# NOTE: Sequential Model provides `predict_classes` or `predict_proba`
#       Functional API Model does not; because it may have multiple outputs
# Using simple `predict` plot instead
prediction_plot(model_combined, (x_test, y_test))

The combine model improves accuracy by 2%, and takes 1/5<sup>th</sup> of the time to train.

#### Training Combining Models

This version of the combined classifier uses both network architectures seen previously; except, in this version, the models need to be trained from scratch.  The following cells repeat the previous experiments with this combined classifier.

*Spoiler:* The combined network doesn't perform any better than the partially trained one did, but takes much longer to train (more epochs).

In [ ]:
batch_size = 128 #32
epochs = 25 #100
learning_rate = 1e-3 #1e-4
decay = 1e-6

model_combined = create_combined_classifier_model()
model_combined.compile(loss='categorical_crossentropy',
                         optimizer=RMSprop(learning_rate=learning_rate, decay=decay),
                         metrics=['accuracy'])
history = model_combined.fit(x_train, y_train,
                             batch_size=batch_size,
                             epochs=epochs,
                             validation_data=(x_test, y_test),
                             shuffle=True)

In [ ]:
history_plot(history)

In [ ]:
# Score trained model.
scores = model_combined.evaluate(x_test, y_test, verbose=0)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

### Skip Connections

From previous comparisons of the `visualize_conv_layer_response` plots of the two basic CNN models, it becomes apparent that the pre-trained VGG16 network contains more complex *knowledge* about images: there were more convolutional layers with a greater variety of patterns and features they could represent.

In the previous cnnbase_classifier model `model_transfer_cnn`, only the last Conv2D layer fed directly to the classifier, and the feature information contained in the middle layers wasn't directly available to the classifier.

Skip Connections are a way to bring lower level feature encodings to higher levels of the network directly.  They are also useful during training very deep networks to deal with the problem of *vanishing gradients*.

In the following example, the original CNN base of the pre-trained VGG16 model is decomposed into layered groups, and a new network created that feeds these intermediate layers to the top of the network, where they are concatenated together to perform the final classification.

* https://towardsdatascience.com/understanding-and-coding-a-resnet-in-keras-446d7ff84d33
* https://arxiv.org/abs/1608.04117

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Concatenate, Flatten, Dense, Activation, Dropout
from tensorflow.keras.applications import VGG16
from tensorflow.keras.optimizers import RMSprop

def create_cnnbase_skipconnected_classifier_model(conv_base=None):
    if not conv_base:
        conv_base = VGG16(weights='imagenet', include_top=False, input_shape=(32, 32, 3))
        conv_base.trainable = False

    # Split conv_base into groups of CNN layers topped by a MaxPooling2D layer
    cb_idxs = [i for (i,l) in enumerate(conv_base.layers) if isinstance(l, keras.layers.MaxPooling2D)]
    all_idxs = [-1] + cb_idxs
    idx_pairs = [l for l in zip(all_idxs, cb_idxs)]
    cb_layers = [conv_base.layers[i+1:j+1] for (i,j) in idx_pairs]

    def dense_classes(l):
        x = Dense(512)(l)
        x = Activation('relu')(x)
        x = Dropout(0.5)(x)
        x = Dense(num_classes)(x)
        return x
    
    inputs = Input(shape=(32,32,3), name='cifar10_image')

    # Join split groups into a sequence, but keep track of their outputs to create skip connections
    skips = []
    inz = inputs
    for lz in cb_layers:
        m = Sequential()
        m.trainable = False
        for ls in lz:
            m.add(ls)
        # inz is the output of model m, but the input for next layer group
        inz = m(inz)
        skips += [inz]

    # Flatten all outputs (which had different dimensions) to Concatenate them on a common axis
    flats = [dense_classes(Flatten()(l)) for l in skips]
    c = Concatenate()(flats)
    x = dense_classes(c)
    outputs = Activation('softmax')(x)

    model = Model(inputs=inputs, outputs=outputs)
    
    return model

In [ ]:
model_skipconnected = create_cnnbase_skipconnected_classifier_model(conv_base)
model_skipconnected.summary()
keras.utils.plot_model(model_skipconnected)

In [ ]:
batch_size = 128 #32
epochs = 25 #100
learning_rate = 1e-3 #1e-4
decay = 1e-6

model_skipconnected.compile(loss='categorical_crossentropy',
                         optimizer=RMSprop(learning_rate=learning_rate, decay=decay),
                         metrics=['accuracy'])
history = model_skipconnected.fit(x_train, y_train,
                                  batch_size=batch_size,
                                  epochs=epochs,
                                  validation_data=(x_test, y_test),
                                  shuffle=True)

In [ ]:
history_plot(history)

In [ ]:
# Score trained model.
scores = model_skipconnected.evaluate(x_test, y_test, verbose=0)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

In [ ]:
# Using simple `predict` plot because model uses Functional API
prediction_plot(model_skipconnected, (x_test, y_test))

### Data Agumentation

Data augmentation is a technique to expand the set of available training data and can significantly improve the performance of image processing networks.

**Note:** Training examples in this section may take significant time. The approach does not improve accuracy result on this simple dataset, but is included here for illustration of the technique.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
    featurewise_center=False,  # set input mean to 0 over the dataset
    samplewise_center=False,  # set each sample mean to 0
    featurewise_std_normalization=False,  # divide inputs by std of the dataset
    samplewise_std_normalization=False,  # divide each input by its std
    zca_whitening=False,  # apply ZCA whitening
    zca_epsilon=1e-06,  # epsilon for ZCA whitening
    rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
    # randomly shift images horizontally (fraction of total width)
    width_shift_range=0.1,
    # randomly shift images vertically (fraction of total height)
    height_shift_range=0.1,
    shear_range=0.1,  # set range for random shear
    zoom_range=0.1,  # set range for random zoom
    channel_shift_range=0.0,  # set range for random channel shifts
    # set mode for filling points outside the input boundaries
    fill_mode='nearest',
    cval=0.0,  # value used for fill_mode = "constant"
    horizontal_flip=True,  # randomly flip images
    vertical_flip=False,  # randomly flip images
    # set rescaling factor (applied before any other transformation)
    rescale=None,
    # set function that will be applied on each input
    preprocessing_function=None,
    # image data format, either "channels_first" or "channels_last"
    data_format=None,
    # fraction of images reserved for validation (strictly between 0 and 1)
    validation_split=0.0
)

# Compute quantities required for feature-wise normalization
# (std, mean, and principal components if ZCA whitening is applied).
datagen.fit(x_train)

In [ ]:
exampledata = datagen.flow(x_train, y_train, batch_size=batch_size)

cifar10_imageset_plot((exampledata[0][0], exampledata[0][1]))

#### CNN Base + Classifier Model Agumented

In [ ]:
batch_size = 128 #32
epochs = 12 #25 #100
learning_rate = 1e-3 #1e-4
decay = 1e-6

model_augmented = create_cnnbase_classifier_model(conv_base)

model_augmented.compile(loss='categorical_crossentropy',
                        optimizer=RMSprop(learning_rate=learning_rate, decay=decay),
                        metrics=['accuracy'])

history = model_augmented.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
                              validation_data=(x_test, y_test),
                              epochs=epochs,
                              shuffle=True,
                              use_multiprocessing=True, workers=4
                             )

In [ ]:
history_plot(history)

In [ ]:
# Score trained model.
scores = model_augmented.evaluate(x_test, y_test, verbose=0)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

#### CNN Classifier Model Augmented

In [ ]:
batch_size = 128 #32
epochs = 12 #25 #100
learning_rate = 1e-3 #1e-4
decay = 1e-6

model_augmented = create_cnn_classifier_model()

model_augmented.compile(loss='categorical_crossentropy',
                        optimizer=RMSprop(learning_rate=learning_rate, decay=decay),
                        metrics=['accuracy'])

history = model_augmented.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
                              validation_data=(x_test, y_test),
                              epochs=epochs,
                              shuffle=True,
                              use_multiprocessing=True, workers=4
                             ) 

In [ ]:
history_plot(history)

In [ ]:
# Score trained model.
scores = model_augmented.evaluate(x_test, y_test, verbose=0)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

#### CNN Base + Skip Connected Classifier Model Agumented

In [ ]:
batch_size = 128 #32
epochs = 12 #25 #100
learning_rate = 1e-3 #1e-4
decay = 1e-6

model_augmented = create_cnnbase_skipconnected_classifier_model(conv_base)

model_augmented.compile(loss='categorical_crossentropy',
                        optimizer=RMSprop(learning_rate=learning_rate, decay=decay),
                        metrics=['accuracy'])

history = model_augmented.fit(datagen.flow(x_train, y_train, batch_size=batch_size),
                              validation_data=(x_test, y_test),
                              epochs=epochs,
                              shuffle=True,
                              use_multiprocessing=True, workers=4
                             )

In [ ]:
history_plot(history)

In [ ]:
# Score trained model.
scores = model_augmented.evaluate(x_test, y_test, verbose=0)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

### Mixed Precision

**TODO**

* https://www.tensorflow.org/guide/keras/mixed_precision
* https://developer.nvidia.com/automatic-mixed-precision
* https://docs.nvidia.com/deeplearning/sdk/mixed-precision-training/index.html

```python
opt = tf.train.experimental.enable_mixed_precision_graph_rewrite(opt) 
```

### Multi-GPU Example

Using multiple GPUs on a single node is a simple way to speed up deep learning.  Keras / TensorFlow support this with a small modification to code.

First, determine if multiple GPUs are available:

In [ ]:
physical_devices = tf.config.experimental.list_physical_devices('GPU')
device_count = len(physical_devices)
print("GPU count:", device_count)
print("GPU devices:", physical_devices)

When scaling to `n` GPUs, there is `n *` the available GPU memory, so we can increase the batch_size by `n`.  A larger batch size means that there is more data evaluated by the batch step, which creates a more accurate and representative loss gradient – so we can take a larger corrective step by multiply the learning_rate by `n`.  Because we are learning `n *` more each epoch, we only need `1/n`<sup>th</sup> the number of training epochs.

There are additional subtleties and mitigating strategies to be aware of when scaling batch sizes larger.  Some of these are discussed in [Deep Learning at scale: Accurate, Large Mini batch SGD](https://towardsdatascience.com/deep-learning-at-scale-accurate-large-mini-batch-sgd-8207d54bfe02).

In [ ]:
# Multi-GPU Example
assert device_count >= 2, "Two or more GPUs required to demonstrate multi-gpu functionality"

from tensorflow.keras.optimizers import Adam, RMSprop
from tensorflow.keras.callbacks import LearningRateScheduler, ReduceLROnPlateau

batch_size = device_count * 128 #32
epochs = 25 // device_count + 1 #100
learning_rate = device_count * 1e-3 #1e-4
decay = 1e-6

def lr_schedule(epoch):
    initial_lr = device_count * 1e-3
    warmup_epochs = 5
    warmup_lr = (epoch + 1) * initial_lr / warmup_epochs
    return warmup_lr if epoch <= warmup_epochs else initial_lr

lr_scheduler = LearningRateScheduler(lr_schedule, verbose=1)
lr_reducer = ReduceLROnPlateau(factor=np.sqrt(0.1), cooldown=0, patience=5, min_lr=0.5e-6)
callbacks = [lr_reducer, lr_scheduler]

strategy = tf.distribute.MirroredStrategy()

with strategy.scope():
    model_multigpu = create_cnnbase_classifier_model()

    model_multigpu.compile(loss='categorical_crossentropy',
                           optimizer=RMSprop(learning_rate=learning_rate, decay=decay, momentum=0.5),
                           # TODO: Explore Adam without lr_scheduling
                           #optimizer=Adam(learning_rate=learning_rate),
                           metrics=['accuracy'])

history = model_multigpu.fit(x_train, y_train,
                             batch_size=batch_size,
                             epochs=epochs,
                             validation_data=(x_test, y_test),
                             shuffle=True,
                             callbacks=callbacks,
                             use_multiprocessing=True, workers=4
                            )

In [ ]:
history_plot(history)

In [ ]:
# Score trained model.
scores = model_multigpu.evaluate(x_test, y_test, verbose=0)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

## Speaker Bios

Glendon Holst is a Staff Scientist in the Visualization Core Lab at KAUST (King Abdullah University of Science and Technology) specializing in HPC workflow solutions for deep learning, image processing, and scientific visualization.

Mohsin Ahmed Shaikh is a Computational Scientist in the Supercomputing Core Lab at KAUST (King Abdullah University of Science and Technology) specializing in large scale HPC applications and GPGPU support for users on Ibex (cluster) and Shaheen (supercomputer).  Mohsin holds a PhD in Computational Bioengineering, and a Post Doc, from University of Canterbury, New Zealand.

## References

* https://qz.com/1034972/the-data-that-changed-the-direction-of-ai-research-and-possibly-the-world/
* https://www.cs.toronto.edu/~kriz/cifar.html
* http://yann.lecun.com/exdb/mnist/index.html
* https://towardsdatascience.com/transfer-learning-from-pre-trained-models-f2393f124751 <br/>
  https://towardsdatascience.com/keras-transfer-learning-for-beginners-6c9b8b7143e
  https://machinelearningmastery.com/how-to-improve-performance-with-transfer-learning-for-deep-learning-neural-networks/
* https://towardsdatascience.com/deep-learning-at-scale-accurate-large-mini-batch-sgd-8207d54bfe02
* https://arxiv.org/abs/1409.1556 <br/>
  https://arxiv.org/abs/1610.02391
* https://www.kaggle.com/c/digit-recognizer
* https://jupyter-notebook.readthedocs.io/en/stable/
* https://github.com/kaust-vislab/handson-ml2
* https://keras.io/examples/cifar10_cnn/ <br/>
  https://keras.io/examples/cifar10_resnet/